In [ ]:
# Install the Kaggle API client
!pip install -q kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d aubird/au-bird-scene-photo-collection

In [ ]:
!unzip au-bird-scene-photo-collection.zip

In [ ]:
import os
import shutil
import random

from xml.etree import ElementTree

In [ ]:
# Parser the data from kaggle

def get_data(filepath):
    all_imgs = {}
    classes_count = {}
    class_mapping = {}

    visualise = True
    i = 1

    for files in os.listdir(filepath):
        if files[-3:] == 'xml':
            filename = os.path.join(filepath, files)
            tree = ElementTree.parse(filename)
            root = tree.getroot()

            width = int(root.find('.//size/width').text)
            height = int(root.find('.//size/height').text)

            all_imgs[filename] = {}
            
            all_imgs[filename]['filename'] = files[:-3] + 'jpg'
            all_imgs[filename]['width'] = width
            all_imgs[filename]['height'] = height
            all_imgs[filename]['bboxes'] = []

            for box in root.findall('.//object'):
                class_name = box.find('name').text
                x1 = int(box.find('.//bndbox/xmin').text)
                y1 = int(box.find('.//bndbox/ymin').text)
                x2 = int(box.find('.//bndbox/xmax').text)
                y2 = int(box.find('.//bndbox/ymax').text)

                all_imgs[filename]['bboxes'].append({'class': class_name, 'x1': x1, 'y1': y1, 'x2': x2, 'y2': y2})

                if class_name not in classes_count:
                    classes_count[class_name] = 1
                else:
                    classes_count[class_name] += 1

                if class_name not in class_mapping:
                    class_mapping[class_name] = len(class_mapping)

    all_data = []

    for key in all_imgs:
        all_data.append(all_imgs[key])

    return all_data, classes_count, class_mapping

In [ ]:
def process_data(filepath, image_folder, label_folder):
    all_imgs, classes_count, class_mapping = get_data(filepath)

    for img in all_imgs:
        # Create image folder
        shutil.copy(os.path.join(filepath, img['filename']), os.path.join(image_folder, img['filename']))

        # Create label folder
        labels = []

        for bbox in img['bboxes']:
            x = (bbox['x2'] + bbox['x1']) / 2
            y = (bbox['y2'] + bbox['y1']) / 2
            w = bbox['x2'] - bbox['x1']
            h = bbox['y2'] - bbox['y1']

            x_normalized = x / img['width']
            y_normalized = y / img['height']
            w_normalized = w / img['width']
            h_normalized = h / img['height']

            labels.append({'x': x_normalized, 'y': y_normalized, 'w': w_normalized, 'h': h_normalized})

        txt_file = img['filename'][:-3] + 'txt'
        with open(os.path.join(label_folder, txt_file), 'w') as f:
            for label in labels:
                f.write('{} {} {} {} {}\n'.format(0, label['x'], label['y'], label['w'], label['h']))

In [ ]:
train_filepath = '/content/AUbirds/train'
val_filepath = '/content/AUbirds/test'

train_image_folder = '/content/birds_data/images/train'
val_image_folder = '/content/birds_data/images/val'

train_label_folder = '/content/birds_data/labels/train'
val_label_folder = '/content/birds_data/labels/val'

process_data(train_filepath, train_image_folder, train_label_folder)
process_data(val_filepath, val_image_folder, val_label_folder)